# TÍNH TOÁN GRADIENT TRONG TORCH.TENSOR (Code for Pytorch)
## Import các Libarary
Sử dụng hai thư viện thao tác với matrix và vector là _numpy_ và _torch_.

In [4]:
import numpy as np
import torch as T
import torch.nn as nn
import torch.nn.functional as F

## Gradient trong Tensor
Thiết lập một bộ số random cố định cho Tensor và tạo một mạng <font color='red'>_NN1_</font> với 2 ngõ vào và một ngõ ra.
_NN1_ có tham số Weigh $\boldsymbol{\theta}=[\theta_1,\theta_2]$ và Bias $b$. Một mini-batch _in_var_ có batch-size $B=3$, $\textbf{X}=\{\textbf{x}_1,\textbf{x}_1,\textbf{x}_3\}$ với $\textbf{x}_n=[x_{n,1},x_{n,2}]^\top, n=1,2,3,$ được đựa vào mạng và hàm _loss_ được tính bởi:
$$
loss_n=NN1(\textbf{x}_n)
$$
Một cách chi tiết hàm _loss_ được viết lại
$$
loss=\theta_1 x_{n,1}+\theta_2 x_{n,2}+b
$$
Như vậy _Gradient_ w.r.t $\boldsymbol{\theta}$, $\nabla_{\boldsymbol{\theta}}|_{\textbf{x}_n}$, chính là $\textbf{x}_n^\top$. Và _Gradient_ cho toàn mini-batch sẽ là trung bình của các _Gradient_ thành phần 
$$
\nabla_{\boldsymbol{\theta}}^\text{mini-batch}=\frac{1}{B}\sum\limits_{n=1}^3 \textbf{x}_n=\left[\frac{1.4}{3},0.2\right]
$$

In [3]:
T.manual_seed(10)
in_var = T.tensor([[0.5,-0.6],[0.7,1.],[0.2,0.2]],requires_grad=True)
NN1 = nn.Linear(2,1)
# nn.init.uniform_(NN1.weight.data,-1,1) # set param with rules
# nn.init.constant_(NN1.bias.data,-1)
out_var = NN1(in_var)

loss = T.mean(out_var)
print(loss)
NN1.zero_grad()
loss.backward()
print(NN1.weight.grad)

tensor(-0.2977, grad_fn=<MeanBackward0>)
tensor([[0.4667, 0.2000]])


## Ảnh hưởng của Detach và Numpy đối với Gradient
Ở phần này _loss_ sẽ được tính toán thủ công thông qua thao tác trên _numpy_ và sau đó kiểm tra _Gradient_ của $\boldsymbol{\theta}$.
Đầu tiên, ta _detach()_ ngõ ra của _NN1_ (chính là _out_var_) sao đó chuyển sang _numpy_. Từ đây _mean_ được tính thủ công. Sau đó, _loss_ được chuyển về dạng _tensor_ tương ứng với dạng _output_ của phép <font color=red>_T.mean(out_var)_</font>. Tương tự ta tính _Gradient_ và thấy $
\nabla_{\boldsymbol{\theta}}^\text{mini-batch}=\left[0,0\right]
$. Như vậy, khi thao tác với _tensor_ phải chú ý việc bảo toàn _Gradient_ ở mỗi bước tính toán.

In [4]:
out_var = out_var.detach()
out_var = out_var.numpy()
lost_1 = out_var.sum()/out_var.shape[0]
lost_1 = T.tensor(lost_1,requires_grad=True)
NN1.zero_grad()
lost_1.backward()
print(NN1.weight.grad)

tensor([[0., 0.]])


## Ảnh hưởng Batch-Norm-1D đối với Gradient
(image from _kratzert.github.io_)<br>
<img src="https://kratzert.github.io/images/bn_backpass/BNcircuit.png" alt="backnorm flow" width="800">

In [25]:
in_var_2 = T.tensor([[1.],[2.],[3.],[4.]],requires_grad=True)
NN2 = nn.Linear(1,1)
BN2 = nn.BatchNorm1d(1)
out_var_2 = BN2(NN2(in_var_2))
print(out_var_2)
loss2 = T.mean(out_var_2)
print(loss2)

NN2.zero_grad()
loss2.backward()
print(NN2.weight.grad)

# print(in_var.shape[0])
# if in_var.shape[0]>1:
#     BN1 = nn.BatchNorm1d(1)
#     out_var_bn = BN1(out_var)
#     print(out_var_bn)
#     out_var_bn = T.mean(out_var_bn)
#     print(out_var_bn)
#     out_var_bn.backward()
#     print(NN1.weight.grad)
    
# pointer_param_NN1  = NN1.named_parameters()
# print(dict(pointer_param_NN1))    
# NN2 = nn.Linear(2,1)
# NN2.load_state_dict(dict(NN1.named_parameters()))
# print(NN1.weight.data)
# print(NN2.weight.data)

# NN2.load_state_dict(dict(NN1.named_parameters()))
# print(NN1.weight.data)
# print(NN2.weight.data)

tensor([[-1.3416],
        [-0.4472],
        [ 0.4472],
        [ 1.3416]], grad_fn=<NativeBatchNormBackward>)
tensor(1.1921e-07, grad_fn=<MeanBackward0>)
tensor([[-1.7513e-07]])


## Phép Toán _TENSOR_ Bảo Toàn Tính _Gradient_ - Hiện Thực Hóa Các _Custom Layer_ Trong _Deep Learning_

### Clone()
Không giống như _copy_()_, hàm _clone()_ được lưu lại để thực hiện tính toán _Gradient_ sau này. Khi thao tác tính _Gradient_, nếu gặp biến _cloned tensor_ python sẽ coi như là biến _original tensor_ và để tính _Gradient_. Như vậy ta có thể viết một _custom layer_ để phân tách các thao tác từng tiểu biến đơn lẻ (tùy thuộc vào bài toán thực tế) của _original tensor_ thông qua _clone_ nó mà đảm bảo _Gradient_ không bị gián đoạn tại bước phân tách này.
### torch.norm(input, p='fro', dim=None, keepdim=False, out=None, dtype=None)
Tính _norm_ cho một _matrix_ hoặc _vector_. Bài toán thực tế thường có những ràng buộc liên quan đến _norm_, vì vậy _torch.norm_ cho phép tính toán _norm_ nhưng bảo toàn tính toán _Gradient_.
#### _p_: loại _norm
- Frobenius norm ($\ell_2$): được tính theo công thức sau
$$
\|\textbf{a}\|_2 = \sqrt{\sum\limits_{n=1}^{N}a_{n}^2}
\\
\|\textbf{A}\|_2 = \sqrt{\sum\limits_{m=1,n=1}^{M,N}a_{m,n}^2}
$$
- _nuclear norm_: (Updating - hình như là norm vô cực)
- _other norm_ ($\ell_n$): là dạng tổng quát
$$
\|\textbf{a}\|_n = \left({\sum\limits_{k=1}^{K}a_{k}^n}\right)^{1/n}
\\
\|\textbf{A}\|_n = \left({\sum\limits_{m=1,k=1}^{M,K}a_{m,k}^n}\right)^{1/n}
$$

#### _dim_: chiều tính _norm_
_dim_ là số nguyên. $dim=0$ và $dim=1$ tính _norm_ theo cột và hàng của _matrix_, có thể hiểu thứ nhất và thứ hai của _multi-dimension matrix_.
### torch.mean()
Tính _mean_ cho một _tensor_ và bảo toàn tính toán _Gradient_.

## Ví Dụ Cho Phép Toán _CLONE(), NORM()_ và _MEAN_
### Mô tả bài toán
Giả sử có một _ouput_ tên _action_ $a$ có dạng $a=[x_{1}^\text{real},x_{2}^\text{real},y^\text{real},z^\text{real},x_{1}^\text{imag},x_{2}^\text{imag},y^\text{imag},z^\text{imag}]$. _output_ đại diện cho 1 vector $\textbf{x}\triangleq[x_1,x_2]$ và 2 biến ngõ ra $y$ và $z$ được xác định bởi
$$
x_1=x_1^\text{real} + 1j*x_1^\text{imag}\\
x_2=x_2^\text{real} + 1j*x_2^\text{imag}\\
y=y^\text{real} + 1j*y^\text{imag}\\
z=z^\text{real} + 1j*z^\text{imag}
$$

Điều kiện ràng buộc ngõ liên quan _norm_ $\ell_2$ và _abs_ như sau
$$
\|\textbf{x}\|_2 = 1\\
|y| = 1\\
|z| = 1
$$
Như vậy, từ _output_ $a$ ta thực hiện tính ràng buộc về _norm_ và _abs_ và thu được _output_ hợp lệ $\hat{a}$. Giả sử hàm _loss_ $f$ có dạng mean của $\hat{a}$
$$f=\mathbb{E}\{\hat{a}\}$$
vậy bước tính đạo hàm của hàm _loss_ theo $a$ liệu có thực hiện được hay không?
### Tính toán
Giá trị của x_1

In [5]:
a = torch.tensor([[1.,2.,3.,4.],[1.,1.,4.,3.]],requires_grad=True)
x = action[:,[0,1]].clone()
yz = action[:,[2,3]].clone()
print('separate var',x,yz)
x_ell2 = torch.norm(GK)
yz_abs = torch.norm(IRS,dim=0)
print('norm value of vector x and abs value of y and z',x_ell2,yz_abs)

x_normed = x/x_ell2
yz_normed = yz/yz_abs
print('vector x after norm: ',x_normed)
print('y and z after norm: ',yz_normed)
a_normed = torch.cat((x_normed,yz_normed),dim=1)
print(a_normed)

# Loss function is mean function
loss = torch.mean(a_normed)
print('Value of loss function: ',loss)
loss.backward()
print('Grad for original action a',a.grad)

NameError: name 'torch' is not defined